In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1001567,2021-02-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1001568,2021-02-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1001569,2021-02-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1001570,2021-02-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27243,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27245,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27247,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27249,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27251,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
621111,2021-02-15,Arkansas,Arkansas,5001,1929,37.00,5000,Arkansas,AR,Arkansas,State,3017804
621113,2021-02-16,Arkansas,Arkansas,5001,1930,38.00,5000,Arkansas,AR,Arkansas,State,3017804
621115,2021-02-17,Arkansas,Arkansas,5001,1933,38.00,5000,Arkansas,AR,Arkansas,State,3017804
621117,2021-02-18,Arkansas,Arkansas,5001,1939,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1001567,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1001568,2021-02-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1001569,2021-02-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1001570,2021-02-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-19') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
395,2021-02-19,Snohomish,Washington,30297,526.00,53061,WA,County,822083,3685.39,63.98
788,2021-02-19,Cook,Illinois,468690,9790.00,17031,IL,County,5150233,9100.36,190.09
1180,2021-02-19,Orange,California,258567,3726.00,6059,CA,County,3175692,8142.07,117.33
1571,2021-02-19,Maricopa,Arizona,503012,8783.00,4013,AZ,County,4485414,11214.39,195.81
1962,2021-02-19,Los Angeles,California,1177340,19662.00,6037,CA,County,10039107,11727.54,195.85
...,...,...,...,...,...,...,...,...,...,...,...
1001175,2021-02-19,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1001305,2021-02-19,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1001404,2021-02-19,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1001499,2021-02-19,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
395,2021-02-19,Snohomish,Washington,30297,526.00,53061,WA,County,822083,3685.39,63.98,63.98,3685.39
788,2021-02-19,Cook,Illinois,468690,9790.00,17031,IL,County,5150233,9100.36,190.09,190.09,9100.36
1180,2021-02-19,Orange,California,258567,3726.00,6059,CA,County,3175692,8142.07,117.33,117.33,8142.07
1571,2021-02-19,Maricopa,Arizona,503012,8783.00,4013,AZ,County,4485414,11214.39,195.81,195.81,11214.39
1962,2021-02-19,Los Angeles,California,1177340,19662.00,6037,CA,County,10039107,11727.54,195.85,195.85,11727.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001175,2021-02-19,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1001305,2021-02-19,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1001404,2021-02-19,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1001499,2021-02-19,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
706469,2021-02-19,Clarke,Alabama,3417,48.00,1025,AL,County,23622,14465.33,203.20,203.20,14465.33,1
649324,2021-02-19,Hale,Alabama,2044,64.00,1065,AL,County,14651,13951.27,436.83,436.83,13951.27,2
508581,2021-02-19,Lowndes,Alabama,1297,46.00,1085,AL,County,9726,13335.39,472.96,472.96,13335.39,3
451151,2021-02-19,Etowah,Alabama,13022,306.00,1055,AL,County,102268,12733.21,299.21,299.21,12733.21,4
378816,2021-02-19,Franklin,Alabama,3954,75.00,1059,AL,County,31362,12607.61,239.14,239.14,12607.61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952854,2021-02-19,Platte,Wyoming,580,11.00,56031,WY,County,8393,6910.52,131.06,131.06,6910.52,19
620135,2021-02-19,Sublette,Wyoming,671,7.00,56035,WY,County,9831,6825.35,71.20,71.20,6825.35,20
855867,2021-02-19,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
769651,2021-02-19,Lincoln,Wyoming,1202,11.00,56023,WY,County,19830,6061.52,55.47,55.47,6061.52,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
508581,2021-02-19,Lowndes,Alabama,1297,46.00,1085,AL,County,9726,13335.39,472.96,472.96,13335.39,3,1
649324,2021-02-19,Hale,Alabama,2044,64.00,1065,AL,County,14651,13951.27,436.83,436.83,13951.27,2,2
449823,2021-02-19,Clay,Alabama,1409,54.00,1027,AL,County,13235,10646.01,408.01,408.01,10646.01,22,3
548298,2021-02-19,Greene,Alabama,858,32.00,1063,AL,County,8111,10578.23,394.53,394.53,10578.23,24,4
507919,2021-02-19,Crenshaw,Alabama,1419,52.00,1041,AL,County,13772,10303.51,377.58,377.58,10303.51,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546977,2021-02-19,Hot Springs,Wyoming,359,3.00,56017,WY,County,4413,8135.06,67.98,67.98,8135.06,16,19
750594,2021-02-19,Uinta,Wyoming,2024,12.00,56041,WY,County,20226,10006.92,59.33,59.33,10006.92,5,20
769651,2021-02-19,Lincoln,Wyoming,1202,11.00,56023,WY,County,19830,6061.52,55.47,55.47,6061.52,22,21
215906,2021-02-19,Teton,Wyoming,3318,9.00,56039,WY,County,23464,14140.81,38.36,38.36,14140.81,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999201,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
999202,2021-02-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
999203,2021-02-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
999204,2021-02-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
703779,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14465.33,45,1,2.00
703780,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14465.33,45,1,0.00
703781,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,203.20,14465.33,45,1,1.00
703782,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,203.20,14465.33,45,1,4.00
703783,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,203.20,14465.33,45,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782415,2021-02-15,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
782416,2021-02-16,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
782417,2021-02-17,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
782418,2021-02-18,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
506543,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13335.39,1,3,1.00,0.00
506544,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13335.39,1,3,0.00,0.00
506545,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13335.39,1,3,0.00,0.00
506546,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13335.39,1,3,0.00,0.00
506547,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13335.39,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504883,2021-02-15,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9979.42,23,6,0.00,0.00
504884,2021-02-16,Albany,Wyoming,3867,11.00,56001,WY,County,38880,9945.99,28.29,28.29,9979.42,23,6,13.00,0.00
504885,2021-02-17,Albany,Wyoming,3871,11.00,56001,WY,County,38880,9956.28,28.29,28.29,9979.42,23,6,4.00,0.00
504886,2021-02-18,Albany,Wyoming,3871,11.00,56001,WY,County,38880,9956.28,28.29,28.29,9979.42,23,6,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-19') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
258800,2021-02-19,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,23.00,0.00
1962,2021-02-19,Los Angeles,California,1177340,19662.00,6037,CA,County,10039107,11727.54,195.85,195.85,11727.54,2,5,2443.00,148.00
452431,2021-02-19,Inyo,California,1213,34.00,6027,CA,County,18039,6724.32,188.48,188.48,6724.32,3,30,16.00,0.00
57881,2021-02-19,Stanislaus,California,54887,921.00,6099,CA,County,550660,9967.49,167.25,167.25,9967.49,4,10,159.00,5.00
71718,2021-02-19,Tulare,California,47399,720.00,6107,CA,County,466195,10167.20,154.44,154.44,10167.20,5,9,88.00,3.00
26892,2021-02-19,Riverside,California,287534,3633.00,6065,CA,County,2470546,11638.48,147.05,147.05,11638.48,6,6,334.00,34.00
410157,2021-02-19,Merced,California,28489,388.00,6047,CA,County,277680,10259.65,139.73,139.73,10259.65,7,8,104.00,4.00
548240,2021-02-19,Kings,California,21797,213.00,6031,CA,County,152940,14251.99,139.27,139.27,14251.99,8,3,77.00,0.00
26192,2021-02-19,Fresno,California,93734,1356.00,6019,CA,County,999101,9381.83,135.72,135.72,9381.83,9,14,179.00,7.00
46085,2021-02-19,San Joaquin,California,65891,1026.00,6077,CA,County,762148,8645.43,134.62,134.62,8645.43,10,18,186.00,5.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
952405,2021-02-19,Lassen,California,5558,19.00,6035,CA,County,30573,18179.44,62.15,62.15,18179.44,36,1,0.00,0.00
258800,2021-02-19,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,23.00,0.00
548240,2021-02-19,Kings,California,21797,213.00,6031,CA,County,152940,14251.99,139.27,139.27,14251.99,8,3,77.00,0.00
129142,2021-02-19,San Bernardino,California,283873,2522.00,6071,CA,County,2180085,13021.19,115.68,115.68,13021.19,13,4,517.00,30.00
1962,2021-02-19,Los Angeles,California,1177340,19662.00,6037,CA,County,10039107,11727.54,195.85,195.85,11727.54,2,5,2443.00,148.00
26892,2021-02-19,Riverside,California,287534,3633.00,6065,CA,County,2470546,11638.48,147.05,147.05,11638.48,6,6,334.00,34.00
162261,2021-02-19,Kern,California,101370,741.00,6029,CA,County,900202,11260.81,82.31,82.31,11260.81,24,7,213.00,14.00
410157,2021-02-19,Merced,California,28489,388.00,6047,CA,County,277680,10259.65,139.73,139.73,10259.65,7,8,104.00,4.00
71718,2021-02-19,Tulare,California,47399,720.00,6107,CA,County,466195,10167.20,154.44,154.44,10167.20,5,9,88.00,3.00
57881,2021-02-19,Stanislaus,California,54887,921.00,6099,CA,County,550660,9967.49,167.25,167.25,9967.49,4,10,159.00,5.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
258800,2021-02-19,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,23.00,0.00
1962,2021-02-19,Los Angeles,California,1177340,19662.00,6037,CA,County,10039107,11727.54,195.85,195.85,11727.54,2,5,2443.00,148.00
452431,2021-02-19,Inyo,California,1213,34.00,6027,CA,County,18039,6724.32,188.48,188.48,6724.32,3,30,16.00,0.00
57881,2021-02-19,Stanislaus,California,54887,921.00,6099,CA,County,550660,9967.49,167.25,167.25,9967.49,4,10,159.00,5.00
71718,2021-02-19,Tulare,California,47399,720.00,6107,CA,County,466195,10167.20,154.44,154.44,10167.20,5,9,88.00,3.00
26892,2021-02-19,Riverside,California,287534,3633.00,6065,CA,County,2470546,11638.48,147.05,147.05,11638.48,6,6,334.00,34.00
410157,2021-02-19,Merced,California,28489,388.00,6047,CA,County,277680,10259.65,139.73,139.73,10259.65,7,8,104.00,4.00
548240,2021-02-19,Kings,California,21797,213.00,6031,CA,County,152940,14251.99,139.27,139.27,14251.99,8,3,77.00,0.00
26192,2021-02-19,Fresno,California,93734,1356.00,6019,CA,County,999101,9381.83,135.72,135.72,9381.83,9,14,179.00,7.00
46085,2021-02-19,San Joaquin,California,65891,1026.00,6077,CA,County,762148,8645.43,134.62,134.62,8645.43,10,18,186.00,5.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3382,02/15/21,Lassen,5498,19.00,17983.19,62.15,62.15,18179.44,36,1,0.00,0.00
3383,02/16/21,Lassen,5546,19.00,18140.19,62.15,62.15,18179.44,36,1,48.00,0.00
3384,02/17/21,Lassen,5546,19.00,18140.19,62.15,62.15,18179.44,36,1,0.00,0.00
3385,02/18/21,Lassen,5558,19.00,18179.44,62.15,62.15,18179.44,36,1,12.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3456,02/15/21,San Joaquin,64969,992.00,8524.46,130.16,134.62,8645.43,10,18,0.00,0.00
3457,02/16/21,San Joaquin,65359,1018.00,8575.63,133.57,134.62,8645.43,10,18,390.00,26.00
3458,02/17/21,San Joaquin,65659,1018.00,8614.99,133.57,134.62,8645.43,10,18,300.00,0.00
3459,02/18/21,San Joaquin,65705,1021.00,8621.03,133.96,134.62,8645.43,10,18,46.00,3.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)